In [ ]:
"""
Data Preparation Pipeline
-------------------------
This script reads product planning data, city–province mappings, 
and invoice records from SQL Server, merges them, converts Jalali 
dates to Gregorian, and performs necessary data cleaning.

Sensitive information has been removed for public sharing.
"""

# 📦 Libraries
import pandas as pd
import jdatetime
from sqlalchemy import create_engine

# =============================
# 🔹 File paths (sanitized)
# =============================
file_path = r"<PATH_TO_PRODUCT_FILE>/DataBase_Product.xlsx"
file_path_1 = r"<PATH_TO_CITY_FILE>/Cities.xlsx"

# =============================
# 🔹 Read Excel files
# =============================
planning_df = pd.read_excel(file_path, sheet_name="Planning")

city_province_df = pd.read_excel(file_path_1)
city_province_df.columns = ['CityName', 'ProvinceName']  # Normalize column names

# =============================
# 🔹 SQL Server connection (sanitized)
# =============================
connection_string = (
    "mssql+pyodbc://<USERNAME>:<PASSWORD>@<SERVER>/<DATABASE>"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

engine = create_engine(connection_string)

# =============================
# 🔹 Read tables from SQL Server
# =============================
query_invoice = "SELECT * FROM dbo.HybridSale"
query_date = "SELECT * FROM dbo.DimDate"

data = pd.read_sql(query_invoice, engine)
df = pd.read_sql(query_date, engine)

# =============================
# 🔹 Jalali → Gregorian converter
# =============================
def jalali_to_gregorian(date_str):
    """Convert 'YYYY/MM/DD' Jalali date to Gregorian (datetime)."""
    try:
        y, m, d = map(int, str(date_str).split('/'))
        g = jdatetime.date(y, m, d).togregorian()
        return pd.to_datetime(g)
    except Exception:
        return pd.NaT

# =============================
# 🔹 Prepare planning data
# =============================
m = planning_df.iloc[:, [0, 4, 10]].copy()
m.columns = ['merge_key', 'BrandName', 'category']

data['ProductCode'] = data['ProductCode'].astype(str)
m['merge_key'] = m['merge_key'].astype(str)

# =============================
# 🔹 Merge planning info into invoices
# =============================
data = data.merge(
    m[['merge_key', 'category']],
    left_on='ProductCode',
    right_on='merge_key',
    how='left'
).drop(columns=['merge_key'])

# Handle duplicate category columns if they appear
if 'category_x' in data.columns and 'category_y' in data.columns:
    data['category'] = data['category_x'].combine_first(data['category_y'])
    data = data.drop(columns=['category_x', 'category_y'])

# Drop rows without category
data = data.dropna(subset=['category'])

# =============================
# 🔹 Add ProvinceName using CityName
# =============================
data = data.merge(
    city_province_df,
    on='CityName',
    how='left'
)

print("✔ ProvinceName column successfully added.")
print(f"✔ Final row count: {len(data):,}")


In [ ]:
"""
RFM Segmentation Pipeline
-------------------------
This script performs RFM calculation, outlier detection, clustering,
segment mapping, new customer identification, and time-range-based
segmentation for customer behavior analysis.

All sensitive information, column names, and paths have been replaced
with placeholders for safe public release on GitHub.
"""

# -------------------------------
# 📦 Imports
# -------------------------------
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans
from scipy.stats import zscore

# ---------------------------------------------------------
# 🔹 Order clusters by the mean of the target variable
# ---------------------------------------------------------
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    df_new["index"] = df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, "index"]], on=cluster_field_name)
    df_final = df_final.drop(columns=[cluster_field_name])
    df_final = df_final.rename(columns={"index": cluster_field_name})
    return df_final

# ---------------------------------------------------------
# 🔹 Segment Map (RFM Score → Segment Name)
# ---------------------------------------------------------
SEGMENT_MAP = {
    "Champions": {"444","443","433","434","343","344","334"},
    "Loyal": {"432","333","324","244","243","234","233","224"},
    "Potential Loyalist": {
        '442','440','441','430','431','422','421','420','341','340','331','330',
        '320','342','322','321','312','242','241','240','231','230','222','212',
        '400'
    },
    "Promising": {
        '414','413','412','411','410','404','403','402',
        '314','313','302','303','304','204','203','202',
        '301','310','311','401'
    },
    "Needs Attention": {'424','423','332','323','232','223','214','213'},
    "About To Sleep": {'220','210','201','110','102','200','300'},
    "At Risk": {
        '144','143','134','133','142','141','132','131',
        '124','123','114','113','042','041','034','032',
        '031','024','023','022','014','013'
    },
    "Critical Loss Risk": {'044','043','033','103','104','004','003','002'},
    "Hibernating": {'221','211','120','130','140','122','121','112','111',
                    '021','012','011','101','100'},
    "Lost": {'000','001','010','020','030','040'}
}

# ---------------------------------------------------------
# 🔹 Assign segment from RFM score
# ---------------------------------------------------------
def assign_segment_from_score(score_str: str) -> str:
    if pd.isna(score_str):
        return None
    for seg, codes in SEGMENT_MAP.items():
        if score_str in codes:
            return seg
    return None

# ---------------------------------------------------------
# 🔹 Remove outliers via Z-Score
# ---------------------------------------------------------
def remove_outliers_zscore(df, cols, threshold=3.0):
    arr = df[cols].to_numpy(dtype=float)
    if arr.shape[0] < 2:
        return df.copy(), pd.DataFrame(), 0
    Z = np.abs(zscore(arr, nan_policy="omit"))
    if len(cols) == 1:
        Z = Z.reshape(-1, 1)
    mask = (Z < threshold) | np.isnan(Z)
    mask = mask.all(axis=1)

    removed_df = df.loc[~mask].copy()
    kept_df = df.loc[mask].copy()
    removed = (~mask).sum()

    return kept_df, removed_df, int(removed)

# ---------------------------------------------------------
# 🔹 Compute RFM for a single period
# ---------------------------------------------------------
def RFM_one_period(df, end_date):
    if df.empty:
        return pd.DataFrame(), pd.DataFrame()

    rfm = (
        df.groupby("CustomerCode")
        .agg(
            # Basic fields
            CustomerName=("CustomerName", "first"),
            CustomerStatus=("CustomerStatus", "first"),
            BranchCode=("BranchCode", "first"),
            BranchName=("BranchName", "first"),
            FactorGDate=("FactorGDate", "first"),
            FactorSDate=("FactorSDate", "first"),
            FactorNumber=("FactorNumber", "first"),
            SupervisorCode=("SupervisorCode", "first"),
            SupervisorName=("SupervisorName", "first"),
            SupervisorStatus=("SupervisorStatus", "first"),
            VisitorCode=("VisitorCode", "first"),
            VisitorName=("VisitorName", "first"),
            VisitorStatus=("VisitorStatus", "first"),
            VisitorRoute=("VisitorRoute", "first"),
            StoreCode=("StoreCode", "first"),
            StoreName=("StoreName", "first"),
            StoreStatus=("StoreStatus", "first"),
            Province=("ProvinceName", "first"),
            CityName=("CityName", "first"),
            CityStatus=("CityStatus", "first"),
            GuildCode=("GuildCode", "first"),
            GuildType=("GuildType", "first"),
            PaymentType=("PaymentType", "first"),
            category=("category", "first"),
            ProductCode=("ProductCode", "first"),
            ProductName=("ProductName", "first"),

            # Sales metrics
            NetCartonQuantity=("NetCartonQuantity", "sum"),
            ReturnCartonQuantity=("ReturnCartonQuantity", "sum"),
            TotalPrice=("TotalPrice", "sum"),
            ReturnTotalPrice=("ReturnTotalPrice", "sum"),

            # RFM
            Recency=("FactorGDate", lambda x: (end_date - x.max()).days),
            Frequency=("FactorNumber", "nunique"),
            Monetary=("TotalPrice", "sum"),
            Returns=("ReturnTotalPrice", "sum"),
        )
        .reset_index()
    )

    # Outlier filtering
    rfm_clean, rfm_outliers, _ = remove_outliers_zscore(
        rfm, ["Recency", "Frequency", "Monetary"], threshold=3.0
    )

    # ------------------------------
    # Safe KMeans wrapper
    # ------------------------------
    def safe_kmeans(df, col, ascending=True):
        n_clusters = min(len(df), 5)
        if n_clusters < 2:
            df[f"{col}Cluster"] = 0
            return df

        km = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
        df[f"{col}Cluster"] = km.fit_predict(df[[col]])
        df = order_cluster(f"{col}Cluster", col, df, ascending=ascending)
        return df

    # --- Clean data clustering ---
    if not rfm_clean.empty:
        rfm_clean = safe_kmeans(rfm_clean, "Recency", ascending=False)
        rfm_clean = safe_kmeans(rfm_clean, "Frequency", ascending=True)

        rfm_clean["NetMonetary"] = (
            rfm_clean["Monetary"] - rfm_clean["Returns"]
        ).clip(lower=0)

        rfm_clean = safe_kmeans(rfm_clean, "NetMonetary", ascending=True)
        rfm_clean = rfm_clean.rename(columns={"NetMonetaryCluster": "MonetaryCluster"})

        rfm_clean["Score"] = (
            rfm_clean["RecencyCluster"].astype(str)
            + rfm_clean["FrequencyCluster"].astype(str)
            + rfm_clean["MonetaryCluster"].astype(str)
        )

        rfm_clean["Segment"] = rfm_clean["Score"].apply(assign_segment_from_score)
        rfm_clean["Date"] = end_date

    # --- Outlier records processing ---
    if not rfm_outliers.empty:
        rfm_outliers = safe_kmeans(rfm_outliers, "Recency", ascending=False)
        rfm_outliers = safe_kmeans(rfm_outliers, "Frequency", ascending=True)

        rfm_outliers["NetMonetary"] = (
            rfm_outliers["Monetary"] - rfm_outliers["Returns"]
        ).clip(lower=0)

        rfm_outliers = safe_kmeans(rfm_outliers, "NetMonetary", ascending=True)
        rfm_outliers = rfm_outliers.rename(columns={"NetMonetaryCluster": "MonetaryCluster"})

        rfm_outliers["Score"] = (
            rfm_outliers["RecencyCluster"].astype(str)
            + rfm_outliers["FrequencyCluster"].astype(str)
            + rfm_outliers["MonetaryCluster"].astype(str)
        )
        rfm_outliers["Segment"] = rfm_outliers["Score"].apply(assign_segment_from_score)
        rfm_outliers["Segment"] = rfm_outliers["Segment"].fillna("Outlier")
        rfm_outliers["Date"] = end_date

    return rfm_clean, rfm_outliers

# ---------------------------------------------------------
# 🔹 Process RFM over custom ranges
# ---------------------------------------------------------
def RFM_over_custom_ranges(data, ranges):
    d = data.copy()
    d["FactorGDate"] = pd.to_datetime(d["FactorGDate"], errors="coerce")

    results, outliers = [], []
    for start_date, end_date in ranges:
        df_period = d[(d["FactorGDate"] >= start_date) & (d["FactorGDate"] <= end_date)]
        if not df_period.empty:
            rfm_period, rfm_outliers = RFM_one_period(df_period, end_date)
            if not rfm_period.empty:
                results.append(rfm_period)
            if not rfm_outliers.empty:
                outliers.append(rfm_outliers)

    final = pd.concat(results, ignore_index=True) if results else pd.DataFrame()
    final_outliers = pd.concat(outliers, ignore_index=True) if outliers else pd.DataFrame()

    return final, final_outliers

# ---------------------------------------------------------
# 🔹 Stabilize segment transitions across time
# ---------------------------------------------------------
def stabilize_segments(rfm_all):
    rfm_all = rfm_all.sort_values(by=["CustomerCode", "Date"])
    rfm_all["FinalSegment"] = None

    allowed_transitions = {
        "Potential Loyalist": {"Loyal","Champions","Promising","Needs Attention"},
        "Loyal": {"Champions","Needs Attention","At Risk"},
        "Champions": {"Loyal","Needs Attention"},
        "Promising": {"Potential Loyalist","Loyal"},
        "Needs Attention": {"At Risk","Hibernating"},
        "At Risk": {"Critical Loss Risk","Hibernating"},
        "Hibernating": {"Lost"},
        "Critical Loss Risk": {"Lost"},
    }

    last_segment = {}

    for idx, row in rfm_all.iterrows():
        cust = row["CustomerCode"]
        seg = row["Segment"]

        if pd.isna(seg):
            rfm_all.at[idx, "FinalSegment"] = "Unknown"
            continue

        if cust not in last_segment:
            rfm_all.at[idx, "FinalSegment"] = seg
            last_segment[cust] = seg
        else:
            prev_seg = last_segment[cust]
            if seg == prev_seg or seg in allowed_transitions.get(prev_seg, set()):
                rfm_all.at[idx, "FinalSegment"] = seg
                last_segment[cust] = seg
            else:
                rfm_all.at[idx, "FinalSegment"] = prev_seg

    rfm_all["FinalSegment"] = rfm_all["FinalSegment"].fillna("Unknown")
    return rfm_all

# ---------------------------------------------------------
# 🔹 Mark newly acquired customers
# ---------------------------------------------------------
def mark_new_customers(rfm_all):
    rfm_all["Date"] = pd.to_datetime(rfm_all["Date"], errors="coerce")

    first_purchases = (
        rfm_all.groupby(["category", "CustomerCode"])["Date"]
        .min()
        .reset_index()
        .rename(columns={"Date": "FirstPurchaseDate"})
    )

    rfm_all = rfm_all.merge(first_purchases, on=["category", "CustomerCode"], how="left")

    rfm_all["Segment"] = np.where(
        rfm_all["Date"].dt.year == rfm_all["FirstPurchaseDate"].dt.year,
        "New Customer",
        rfm_all["Segment"],
    )

    return rfm_all.drop(columns=["FirstPurchaseDate"])

# ----------------------------
# 🔹 Start-date filtering
# ----------------------------
start_filter_fixed = datetime(2024, 3, 20)
df["MiladiDate"] = pd.to_datetime(df["MiladiDate"])
df = df[df["MiladiDate"] >= start_filter_fixed]

# ----------------------------
# 🔹 Build date ranges per YearMonth
# ----------------------------
ranges = (
    df.groupby("YearMonthName")["MiladiDate"]
    .agg(["min", "max"])
    .reset_index()
    .sort_values("min")
)

custom_ranges = [(row["min"], row["max"]) for _, row in ranges.iterrows()]

# ----------------------------
# 🔹 Final Execution
# ----------------------------
output_file = "RFM_custom_ranges.xlsx"
all_results, all_outliers = [], []

for val in data["category"].dropna().unique():
    subset = data[data["category"] == val]

    rfm_result, outliers_result = RFM_over_custom_ranges(subset, custom_ranges)
    rfm_result = stabilize_segments(rfm_result)
    rfm_result = mark_new_customers(rfm_result)

    if not rfm_result.empty:
        rfm_result["Date"] = pd.to_datetime(rfm_result["Date"]).dt.date

    all_results.append(rfm_result)
    if not outliers_result.empty:
        all_outliers.append(outliers_result)

# ----------------------------
# 🔹 Merge Normal + Outliers
# ----------------------------
final_output = (
    pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()
)

final_outliers = (
    pd.concat(all_outliers, ignore_index=True) if all_outliers else pd.DataFrame()
)

if not final_outliers.empty:
    final_outliers["OutlierFlag"] = "Outlier"
    final_output["OutlierFlag"] = "Normal"
    final_output = pd.concat([final_output, final_outliers], ignore_index=True)
else:
    final_output["OutlierFlag"] = "Normal"

# Drop previous FinalSegment if exists
if "FinalSegment" in final_output.columns:
    final_output = final_output.drop(columns=["FinalSegment"])

# ----------------------------
# 🔹 Export to Excel
# ----------------------------
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    final_output.to_excel(writer, sheet_name="RFM_Data", index=False)

print(f"✔ Output saved to '{output_file}' in sheet 'RFM_Data'")
